https://www.kaggle.com/code/hakanerdem/recommender-system-with-artificial-neural-networks

# Intro

Recommender systems can be used to evaluate cross selling opportunities on the domain of retail marketing. Further reading about the key concepts of "recommender systems",  "cross selling", "collaborative filtering" and "deep learning" and their applications, one can look related papers such as:

1. Kamakura, W. A., Wedel, M., de Rosa, F., Mazzon, J. A. (2003), Cross-selling through database marketing: A mixed data factor analyzer for data augmentation and prediction. International Journal of Research in Marketing, 20, 45–65.

2. Knott, A., Hayes, A., & Neslin, S. A. (2002), Next-product-to-buy models for cross-selling applications. Journal of Interactive Marketing, 16(3), 59–75.

3. Thuring F., Nielsen J.P., Guillén M., Bolancé C.,(2012), Selecting prospects for cross-selling ﬁnancial products using multivariate credibility, Expert Systems with Applications 39, 8809–8816.

4. Zhang S., Yao L., Sun A., Tay Y., (2018), Deep Learning based Recommender System: A Survey and New Perspectives. ACM Comput. Surv. 1(1), 1-35.

5. Shi Y., Larson M., Hanjalic A., (2014), Collaborative filtering beyond the user-item matrix: A survey of the state of the art and future challenges. ACM Comput. Surv. 47(1) 45. DOI: http://dx.doi.org/10.1145/2556270

6. Hidasi B., Karatzoglou A., (2018), Recurrent Neural Networks with Top-k Gains for Session-based Recommendations. In The 27th ACM International Conference on Information and Knowledge Management (CIKM ’18), October 22–26, 2018, Torino, Italy. ACM, New York, NY, USA, 10 pages. DOI: https://doi.org/10.1145/3269206.3271761

.
.

In this notebook, we, mainly aimed at prediction of customers next products to buy using implicit feedback from purchase preferences, and mainly follow [Lazy Programmer Inc.'s](https://www.udemy.com/course/recommender-systems/) -which is a udemy course about recommender systems that we strongly recommend- methodology.

Dataset choosen, famous, Online Retail II. For detailed information please visit:

[https://www.kaggle.com/mashlyn/online-retail-ii-uci](https://www.kaggle.com/mashlyn/online-retail-ii-uci)

and for the original source:

[UCI Repository](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II)

# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.version.VERSION)
print(keras.__version__)

# Data & Preparation

In [ ]:
data = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv",
                   parse_dates=["InvoiceDate"],
                   dtype={"Customer ID":"object"})

In [ ]:
df = data.copy()
df.head()

## A brief of data cleaning

Droping rows with missing values and irrelevant labels

In [ ]:
df = df.dropna()
df = df.drop(df[df["Quantity"]<0].index)

In [ ]:
df["StockCode"].str.len().value_counts()

In [ ]:
df = df.drop(df[((df["StockCode"].str.len()>7) | (df["StockCode"].str.len()<5))].index)
df["StockCode"] = df["StockCode"].apply(lambda x: x[:5])
df = df.drop(df[df["StockCode"].str.contains(r"[A-Za-z]")].index)

In [ ]:
grouped_df = df.groupby(["Customer ID", "StockCode"], as_index=False)["Quantity"].sum()
new_df = grouped_df.astype(float).astype(int).rename(columns={"Customer ID": "user_id", "StockCode": "item_id", "Quantity": "purchase"})

By following cells, we try to generate customers' purchase sequences of distinct products. 

We choose some hyperparameter values arbitrarily but it can be a good practice to look at some statistics like below: number of distinct products purchased.  

## Negative Sampling

Since all instances prepared so far represent positive-only feedback, we try to supply some negative information to the model. Negative instances are chosen from products not purchased for a particular customer.
> sample_size=1 

means there is 1 non-purchased product to be selected randomly.

In [ ]:
def neg(x, corp, sample_size=1):
    diff = np.setdiff1d(corp, list(x))
    ind = np.random.permutation(len(diff))
    return diff[ind[:int(sample_size*len(x))]]

corp = new_df["item_id"].unique()
ndf = new_df.groupby("user_id", as_index=False).agg({"item_id": (lambda x: list(neg(x, corp, 1)))})
ndf = ndf.explode("item_id")
ndf["purchase"] = 0
sample_df = pd.concat([new_df, ndf], axis=0)

In [ ]:
VAL_SIZE = 0.2

val_users = sample_df["user_id"].drop_duplicates().sample(frac=VAL_SIZE)

sample_df = sample_df.sample(len(sample_df))

target_item = sample_df.groupby("user_id").agg({"item_id": lambda x: list(x)[-1]})
target_item.rename(columns={"item_id":"target_id"}, inplace=True)

train_item = sample_df.groupby("user_id").agg({"item_id": lambda x: list(x)[:-1]})

target_purchase = sample_df.groupby("user_id").agg({"purchase": lambda x: list(x)[-1]})
target_purchase.rename(columns={"purchase":"target"}, inplace=True)

train_purchase = sample_df.groupby("user_id").agg({"purchase": lambda x: list(x)[:-1]}).explode("purchase")

train_data = pd.concat([target_item, train_item, target_purchase], axis=1).explode("item_id")
train_data = pd.concat([train_data, train_purchase], axis=1).reset_index()

train_set = train_data[["user_id", "item_id", "purchase"]]
train_set = train_set.astype(int)
val_data = train_data[["user_id", "target_id", "target"]].drop_duplicates()
val_set = val_data[val_data["user_id"].isin(val_users)]

print("total users:", train_set["user_id"].nunique())
print("validation users:", val_set["user_id"].nunique())

## Encoding

As a last step we try to encode user and product features. Method taken from [keras.io](https://keras.io/examples/structured_data/collaborative_filtering_movielens/) examples. We take the data as train & validation, but the better practice is holding out some samples in advance as test data.    

In [ ]:
user_ids = train_set["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = sample_df["item_id"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
train_set["user_id"] = train_set["user_id"].map(user2user_encoded)
train_set["item_id"] = train_set["item_id"].map(item2item_encoded)
val_set["user_id"] = val_set["user_id"].map(user2user_encoded)
val_set["target_id"] = val_set["target_id"].map(item2item_encoded)

train_set["purchase"] = train_set["purchase"].apply(lambda x: 1 if x>0 else 0)
val_set["target"] = val_set["target"].apply(lambda x: 1 if x>0 else 0)

# Neural Collaborative Filtering

Neural Collaborative Filtering (NCF) is one of the recommendation system frameworks, based on neural networks, proposed by He, et. al. (2017). According to them a neural network can develop a model by learning item user interactions as a key factor of a collaboritive filtering from implicit feedback. Python code mostly developed thanks to beautiful notebooks like:

[fuzzywizard](https://www.kaggle.com/fuzzywizard/rec-sys-collaborative-filtering-dl-techniques#4-Matrix-Factorization-using-Deep-Learning-(Keras))

[rajmehra03](https://www.kaggle.com/rajmehra03/cf-based-recsys-by-low-rank-matrix-factorization)

Please see for detailed information about NCF:

He, X., Liao, L., Zhang, H., Nie, L., Hu, X.,Chua, T.,  (2017), Neural Collaborative Filtering. WWW'17: Proceedings of the 26th International Conference on World Wide Web 173–182 DOI: http://dx.doi.org/10.1145/3038912.3052569

## Generalized Matrix Factorization

In the first part, we try to implement a collaborative filtering system using embedding layers for user-item instances. Python code mostly adapted from the notebooks of:   

* [colinmorris-1](https://www.kaggle.com/colinmorris/embedding-layers)
* [colinmorris-2](https://www.kaggle.com/colinmorris/matrix-factorization)
* [rajmehra03-1](https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer)
* [rajmehra03-2](https://www.kaggle.com/rajmehra03/cf-based-recsys-by-low-rank-matrix-factorization)
* [rounakbanik](https://www.kaggle.com/rounakbanik/movie-recommender-systems)
* [keras.io examples](https://keras.io/examples/structured_data/collaborative_filtering_movielens/)

and we try to evaluate the system based on the instructions from: 

[jamesloy](https://www.kaggle.com/jamesloy/deep-learning-based-recommender-systems)



In [ ]:
num_users = train_set["user_id"].nunique()
num_items = train_set["item_id"].nunique()
embedding_size = 64

user_input = layers.Input(shape=(1,), name="user_id", dtype=tf.int32)
item_input = layers.Input(shape=(1,), name="item_id", dtype=tf.int32)
 
user_embedding = layers.Embedding(num_users,
                                  embedding_size,
                                  name="user_emb")(user_input)
user_bias = layers.Embedding(num_users, 1, name="user_bias")(user_input)
 
item_embedding = layers.Embedding(num_items,
                                  embedding_size,
                                  name="item_emb")(item_input)
item_bias = layers.Embedding(num_items, 1, name="item_bias")(item_input)
 
user_vector = layers.Flatten()(user_embedding)
item_vector = layers.Flatten()(item_embedding)
 
dot_user_item = layers.Dot(name="dot", axes=1)([user_vector, item_vector])
output = layers.Add(name="add")([dot_user_item, user_bias, item_bias])
output = layers.Flatten(name="flat")(output)
 
model_X = keras.Model([user_input, item_input], output, name="model_x")
 
model_X.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                metrics=["mae"])
 
model_X.summary()

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   verbose=1,
                                   patience=3)

history = model_X.fit([train_set["user_id"].values, train_set["item_id"].values],
                      train_set["purchase"].values,
                      batch_size=256,
                      epochs=200,
                      verbose=1,
                      validation_data=([val_set["user_id"].values, val_set["target_id"].values], val_set["target"].values),
                      callbacks=[es])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("embedding loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="best")
plt.show()

## Multi-Layer Perceptron

In [ ]:
hidden_units = [128, 64]
 
user_input = layers.Input(shape=(1,), name="user_id", dtype=tf.int32)
item_input = layers.Input(shape=(1,), name="item_id", dtype=tf.int32)
user_embedding = layers.Embedding(num_users,
                                  embedding_size,
                                  name="user_emb")(user_input)
item_embedding = layers.Embedding(num_items,
                                  embedding_size,
                                  name="item_emb")(item_input)

concatenated = layers.Concatenate(name="concat")([user_embedding, item_embedding])
out = layers.Flatten(name="flat")(concatenated)
 
for n_hidden in hidden_units:
    out = layers.Dense(n_hidden,
                       activation="relu")(out)
    out = layers.Dropout(0.4)(out)
    out = layers.BatchNormalization()(out)

out = layers.Dense(1, activation="sigmoid", name="prediction")(out)
 
model_Y = keras.Model(inputs = [user_input, item_input],
                      outputs = out, name="model_y")
 
model_Y.compile(loss=keras.losses.MeanSquaredError(),
                optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                metrics=["mae"])
 
model_Y.summary()

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   verbose=1,
                                   patience=3)

history = model_Y.fit([train_set["user_id"].values, train_set["item_id"].values],
                      train_set["purchase"].values,
                      batch_size=256,
                      epochs=200,
                      verbose=1,
                      validation_data=([val_set["user_id"].values, val_set["target_id"].values], val_set["target"].values),
                      callbacks=[es])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("mlp loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="best")
plt.show()

## Neural Matrix Factorization

In [ ]:
hidden_units = [128, 64]

user_input = layers.Input(shape=(1,), name="user_id", dtype=tf.int32)
item_input = layers.Input(shape=(1,), name="item_id", dtype=tf.int32)
user_embedding = layers.Embedding(num_users,
                                  embedding_size,
                                  name="user_emb")(user_input)
user_bias = layers.Embedding(num_users, 1, name="user_bias")(user_input)
 
item_embedding = layers.Embedding(num_items,
                                  embedding_size,
                                  name="item_emb")(item_input)
item_bias = layers.Embedding(num_items, 1, name="item_bias")(item_input)

user_vector = layers.Flatten()(user_embedding)
item_vector = layers.Flatten()(item_embedding)

dot_user_item = layers.Dot(name="dot", axes=1)([user_vector, item_vector])
X = layers.Add(name="add")([dot_user_item, user_bias, item_bias])
X = layers.Flatten(name="flat1")(X)

concatenated = layers.Concatenate(name="concat")([user_embedding, item_embedding])
Y = layers.Flatten(name="flat2")(concatenated)
 
for n_hidden in hidden_units:
    Y = layers.Dense(n_hidden,
                     activation="relu")(Y)
    Y = layers.Dropout(0.4)(Y)
    Y = layers.BatchNormalization()(Y)

Y = layers.Dense(1, activation="sigmoid", name="prediction")(Y)

Z = layers.Add(name="final")([X, Y])

model_Z = keras.Model(inputs = [user_input, item_input],
                      outputs = Z, name="model_z")
 
model_Z.compile(loss=keras.losses.MeanSquaredError(),
                optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                metrics=["mae"])
 
model_Z.summary()

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   verbose=1,
                                   patience=3)

history = model_Z.fit([train_set["user_id"].values, train_set["item_id"].values],
                      train_set["purchase"].values,
                      batch_size=256,
                      epochs=200,
                      verbose=1,
                      validation_data=([val_set["user_id"].values, val_set["target_id"].values], val_set["target"].values),
                      callbacks=[es])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("ncf loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="best")
plt.show()

# Model Evaluation

We try to measure the model performance by providing candidate products to the model and evaluating the outputs. Candidate products are merged with 49 products selected from non-purchased products and a target product which respresented in output_sequence variable. If target product occures in the top k of the model outputs, we count this event as a hit.

On the other hand; Hidasi and Karatzoglou (2018) define "recall@k" as an evaluatinon metric as "the proportion of cases having the desired item amongst the top-k items in all test cases." Moreover, one another evaluation metric is "MRR@k", which is the average of reciprocal ranks of the target items. The reciprocal rank is set to zero if the rank is above k.

In [ ]:
# def get_metrics(model, k, size=1000):  
#     hit = 0
#     mrr = 0
#     counter = size
#     for s in range(counter):
#         cust_id = sample_df["CustomerID"].unique()[s]
#         cust_encoder = cust2cust_encoded.get(cust_id)
#         purchased = frequent_df[(frequent_df.index==cust_id) & (frequent_df["purchase"]==1)]
#         candidates = frequent_df[~frequent_df["input_sequence"].isin(purchased["input_sequence"].values)]["input_sequence"][:49]
#         candidates = set(candidates).intersection(set(prod2prod_encoded.keys()))
#         candidates = candidates.union(set(frequent_df[frequent_df.index==cust_id]["output_sequence"].values[0]))
#         candidates = [[prod2prod_encoded.get(x)] for x in list(candidates)]
#         ids = np.stack([[cust_encoder]]*len(candidates))
#         y_pred = model.predict([ids, np.array(list(candidates), dtype="int32")]).flatten()
#         t = frequent_df.loc[(frequent_df.index==cust_id), "output_sequence"].values[0][0]
#         recommend = []
#         rr = 0
#         for i in range(k):
#             p = prod_encoded2prod.get(candidates[y_pred.argsort()[-(i+1)]][0])
#             recommend.append(p)
#             if (p==t):
#                 rr = 1/(i+1)
#                 hit = hit + 1
#         mrr = mrr + rr
        
#     return (hit/counter), (mrr/counter)

In [ ]:
# recallx, mrrx = get_metrics(model_X, 10)
# print("Recall@: ", recallx)
# print("MRR@: ", mrrx)

In [ ]:
# recally, mrry = get_metrics(model_Y, 10)
# print("Recall@: ", recally)
# print("MRR@: ", mrry)

In [ ]:
# recallz, mrrz = get_metrics(model_Z, 10)
# print("Recall@: ", recallz)
# print("MRR@: ", mrrz)

# Critiques

Please criticise this study and faulty issues other than hyperparameter tuning. Any comment is more precious than upvotes for this fresh notebook. To compare metrics please see: https://medium.com/decathlondevelopers/building-a-rnn-recommendation-engine-with-tensorflow-505644aa9ff3. They developed a model for more than 10,000 different products. 

Some hyperparameters which should be tuned.

* Number and unit numbers of *hidden_units*
* *prod_embedding_size* and *user_embedding_size*
* regularizers, learning rate, activation functions, batch size, dropout rates
* *n_frequency* number of frequent products
* *sample_size* number of negative samples corresponding to a positive sample

Some topics which are ambiguous:

* Can prediction performance be upgraded for this model? 
* Is there a need for negative sampling? Is there a room for improvement by adjusting negative sample size?
* Are there more suitable or effective techniques to measure the performance of the model?
* Can execution time be shortened?
* Any other effective ways to predict next-product-to-buy using deep learning?
.
.

Sorry for language...
Thanks in advance